In [1]:
# Imports

In [1]:
import glob
import os 
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [3]:
# Mean square error

In [2]:
def mean_squared_loss(x1,x2):
    diff=x1-x2
    a,b,c,d,e=diff.shape
    n_samples=a*b*c*d*e
    sq_diff=diff**2
    Sum=sq_diff.sum()
    dist=np.sqrt(Sum)
    mean_dist=dist/n_samples

    return mean_dist

In [5]:
# Process Video Train and Test Video Data

In [3]:
from keras.preprocessing.image import img_to_array,load_img
from PIL import Image
from scipy.misc import imresize

Using TensorFlow backend.


In [7]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def remove_old_images(path):
    filelist = glob.glob(os.path.join(path, "*.png"))
    for f in filelist:
        os.remove(f)

def store(image_path,imagestore):
    img=load_img(image_path)
    img=img_to_array(img)

    #Resize the Image to (227,227,3) for the network to be able to process it. 
    img=imresize(img,(227,227,3))
    #img = img.resize((227,227,3))

    #Convert the Image to Grayscale
    gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]
    imagestore.append(gray)
    
def create(path,fps,name,imagestore=[]):
    #List of all Videos in the Source Directory.
    videos=os.listdir(path)
    print("Found ",len(videos)," video")
    #Make a temp dir to store all the frames
    create_dir(path+'/frames')

    #Remove old images
    remove_old_images(path+'/frames')
    framepath=path+'/frames'

    for video in tqdm(videos):
        os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(path,video,fps,path))
        images=os.listdir(framepath)
        for image in images:
            image_path=framepath+ '/'+ image
            store(image_path,imagestore)

    imagestore=np.array(imagestore)
    a,b,c=imagestore.shape

    #Reshape to (227,227,batch_size)
    imagestore.resize(b,c,a)
    #Normalize
    imagestore=(imagestore-imagestore.mean())/(imagestore.std())
    #Clip negative Values
    imagestore=np.clip(imagestore,0,1)
    np.save(name,imagestore)
    #Remove Buffer Directory
    os.system('rm -r {}'.format(framepath))

In [8]:
# Train Data 

In [9]:
video_path= "./Dataset/training_videos"
# --> FPS
fps = 5
imagestore=[]
create(video_path,fps,'training.npy',imagestore)

Found  16  video


D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  from ipykernel import kernelapp as app


In [10]:
video_path= "./Dataset/testing_videos"
# --> FPS
fps = 5
imagestore=[]
create(video_path,fps,'testing.npy',imagestore)

Found  21  video


D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  from ipykernel import kernelapp as app


In [11]:
# Model 1

In [12]:
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential


def load_model():
    
    model=Sequential()
    model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
    model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))


    model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
    model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
    model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))

    model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
    model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

    model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

    return model

In [13]:
model = load_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# ================================ Train 

In [14]:
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [15]:
X_train=np.load('training.npy')
frames=X_train.shape[2]

frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()

epochs=80
batch_size=1

callback_early_stopping = EarlyStopping(monitor='loss', patience=3)
callback_save = ModelCheckpoint("model.h5", monitor="mean_squared_error", save_best_only=True)

model.fit(X_train,Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0, callbacks = [TQDMNotebookCallback(leave_inner=True,leave_outer=True),callback_early_stopping]
          )

model.save('model.h5')

Instructions for updating:
Use tf.cast instead.


In [4]:
from keras.models import load_model
model = load_model("model.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [8]:
threshold=0.00051
X_test=np.load('testing.npy')
frames=X_test.shape[2]

flag=0 #Overall video flagq
frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)

for number,bunch in enumerate(X_test):
    n_bunch=np.expand_dims(bunch,axis=0)
    reconstructed_bunch=model.predict(n_bunch)


    loss=mean_squared_loss(n_bunch,reconstructed_bunch)

    if loss>threshold:
        print("Anomalous bunch of frames at bunch number {}".format(number))
        flag=1
    else:
        print('Bunch Normal')


if flag==1:
    print("Anomalous Events detected")

Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal


In [18]:
# Camera

In [19]:
from keras.models import load_model

In [20]:
#model = load_model('model.h5')

In [5]:
import cv2
%matplotlib inline

vc=cv2.VideoCapture(1)
rval=True

#threshold=0.0005
threshold=0.00051
text = ""
while rval:
    imagedump=[]
    for i in range(10):
        ok_flag,frame=vc.read()
        cv2.putText(frame, text, (250, 50), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 0, 255))
        cv2.imshow("Camera", frame)
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            rval = False
            
        if cv2.waitKey(1) == 27:
            rval = False
            
        frame=imresize(frame,(227,227,3))

        #Convert the Image to Grayscale

        gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray=(gray-gray.mean())/gray.std()
        gray=np.clip(gray,0,1)
        imagedump.append(gray)

    imagedump=np.array(imagedump)

    imagedump.resize(227,227,10)
    imagedump=np.expand_dims(imagedump,axis=0)
    imagedump=np.expand_dims(imagedump,axis=4)


    #print('Processing data')
    text = ""

    output=model.predict(imagedump)
    loss=mean_squared_loss(imagedump,output)


    if loss>threshold:
        text = "Anomalies Detected"
        print('Anomalies Detected')

cv2.destroyAllWindows()

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Anomalies Detected
Anomalies Detected
Anomalies Detected
Anomalies Detected
